# CONVERT FILES WITH DUCK_DB

In [3]:
import pandas as pd
import duckdb
import os
import time
import numpy as np

In [4]:
def time_it(func):
    """Decorator to measure the execution time of a function."""
    def wrapper(*args, **kwargs):
        start_time = time.time()  
        result = func(*args, **kwargs) 
        end_time = time.time() 
        elapsed_time = end_time - start_time  
        print(f"Execution time: {elapsed_time:.4f} seconds")  
        return result
    return wrapper

## Ownership history

In [5]:
balkans = [
    "BA", "XK", "MK", "CS", "AL"
]
##

In [42]:

TEMP_TABLE_FIRMOGRAPHICS = "..\\data_processed\\firmographics_processed\\*.parquet" 
TEMP_TABLE_KEY_FINANCIALS = "..\\data_processed\\firmographics_historicals_processed"


def get_ownership_data(year, country, path=None):
    print(f"{country} - {year}...")
    COUNTRY_KEY_FINANCIALS = TEMP_TABLE_KEY_FINANCIALS + f"\\firmographics_historicals_{country}.parquet"

    query = f"""
        SELECT
            main.subsidiary_bvd_id,
            main.guo_25,
            firmographics_sub.nuts2 AS subsidiary_nuts2,
            firmographics_sub.nace_rev_2_core_code_4_digits_ AS subsidiary_nace4,
            firmographics_guo.nuts2 AS guo_nuts2,
            firmographics_guo.nace_rev_2_core_code_4_digits_ AS guo_nace4,
            firmographics_guo.type_of_entity AS guo_type_of_entity,
            firmographics_guo.status AS guo_status,
            {year} AS year,
            key_financials.number_of_employees,
            key_financials.closing_date,
            key_financials.operating_revenue_turnover_,
        FROM 
            '{path}' AS main
        LEFT JOIN 
            '{TEMP_TABLE_FIRMOGRAPHICS}' AS firmographics_sub
        ON 
            main.subsidiary_bvd_id = firmographics_sub.bvd_id_number
        LEFT JOIN
            '{TEMP_TABLE_FIRMOGRAPHICS}' AS firmographics_guo
        ON
            main.guo_25 = firmographics_guo.bvd_id_number
        LEFT JOIN 
            (
            SELECT
                bvd_id_number,
                year,
                number_of_employees,
                closing_date,
                operating_revenue_turnover_ 
            FROM '{COUNTRY_KEY_FINANCIALS}'
            WHERE 
                year = {year}
            ) AS key_financials
        ON
            main.subsidiary_bvd_id = key_financials.bvd_id_number   -- CHANGE HERE FOR SUBS
        AND
            key_financials.year = {year}

        WHERE 
            main."type_of_relation" = 'GUO 25'
        AND 
            main."subsidiary_bvd_id" LIKE '{country}%'
    """

    conn = duckdb.connect()
    df = conn.execute(query).fetchdf()
    return df



In [51]:
al = pd.read_parquet(TEMP_TABLE_KEY_FINANCIALS + f"\\firmographics_historicals_AL.parquet")

In [58]:
# path = "..\\data_raw\\ownership_history\\links_2007\\*.parquet"
country = "AL"
year = "2019"
path = "..\\data_raw\\ownership_history\\links_2019\\*.parquet"

df_al = get_ownership_data(
    path=path,
    year=year,
    country=country,
)
# fetch_and_convert_to_excel(country=country, year=year, path=path, output_path=output_path)

AL - 2019...


In [43]:
@time_it
def fetch_and_convert_to_excel(year, country, path, output_path):
    df = get_ownership_data(year, country, path)
    os.makedirs(output_path, exist_ok=True)

    # Split the DataFrame into 5 chunks
    chunks = 5
    indices = np.array_split(df.index, chunks)
    slices = [df.iloc[idx] for idx in indices]

    for i, s in enumerate(slices):
        s.to_csv(f"{output_path}\\{country}_{year}_chunk_{i+1}.csv", index=False)
    
        # s.to_stata(f"{output_path}\\{country}_{year}_chunk_{i+1}.dta", write_index=False)
        print(f"Data for {country} - {year} - slice_{i+1} has been converted.")
        

### Test

In [44]:
# path = "..\\data_raw\\ownership_history\\links_2007\\*.parquet"
country = "AL"
year = "2009"
path = "..\\data_raw\\ownership_history\\links_2007\\*.parquet"

df = get_ownership_data(
    path=path,
    year=year,
    country=country,
)
# fetch_and_convert_to_excel(country=country, year=year, path=path, output_path=output_path)

AL - 2009...


In [48]:
df.operating_revenue_turnover_.value_counts()

operating_revenue_turnover_
94873392.0    4
96233794.0    4
14258341.0    4
32640104.0    4
Name: count, dtype: int64

In [8]:
import time
import json

errors = {}
OUTPUT_PATH = r"..\data_processed\\guo_subs_europee\\subs_eu\\" 

total_start = time.time()
start_time = time.time()


for country in balkans:
    for year in range(2007, 2023):
        year = str(year)
        try:
            path = f"..\\data_raw\\ownership_history\\links_{year}\\*.parquet"
            fetch_and_convert_to_excel(year, country, path, OUTPUT_PATH)  
            end_time = time.time()
            duration = end_time - start_time
            print(f"Data for {country} - {year} has been converted to Stata in {duration:.2f} seconds.")
        except Exception as e:
            errors[f"{country} - {year}"] = str(e)
            print(f"Error {country} - {year}")
            continue


end_time = time.time()
duration = end_time - start_time
total_end = time.time()
total_duration = total_end - total_start
print(f"Total execution time: {total_duration:.2f} seconds.")

with open("errors_guos.json", "w") as f:
    json.dump(errors, f)

AT - 2022...
Data for AT - 2022 - slice_1 has been converted.
Data for AT - 2022 - slice_2 has been converted.
Data for AT - 2022 - slice_3 has been converted.
Data for AT - 2022 - slice_4 has been converted.
Data for AT - 2022 - slice_5 has been converted.
Execution time: 95.8373 seconds
Data for AT - 2022 has been converted to Stata in 95.84 seconds.
BE - 2022...
Data for BE - 2022 - slice_1 has been converted.
Data for BE - 2022 - slice_2 has been converted.
Data for BE - 2022 - slice_3 has been converted.
Data for BE - 2022 - slice_4 has been converted.
Data for BE - 2022 - slice_5 has been converted.
Execution time: 396.7448 seconds
Data for BE - 2022 has been converted to Stata in 492.58 seconds.
BG - 2022...
Data for BG - 2022 - slice_1 has been converted.
Data for BG - 2022 - slice_2 has been converted.
Data for BG - 2022 - slice_3 has been converted.
Data for BG - 2022 - slice_4 has been converted.
Data for BG - 2022 - slice_5 has been converted.
Execution time: 151.8585 secon

In [9]:
# open errors
import json 

with open("errors_guos.json", "r") as f:
    errors = json.load(f)

errors

{}

In [10]:
# import shutil

# ZIP_PATH = r"..\data_processed\\guo_subs_europee\\subs_eu"
# ZIP_FOLDER = r"..\data_processed\\guo_subs_europee"

# shutil.make_archive(
#     base_name=fr"{ZIP_FOLDER}\subs_eu_zipped",  
#     format="zip",
#     base_dir=ZIP_PATH  
# )